# Pandas

In [17]:
import pandas as pd

## Load data

In [118]:
DATA_DIR = "../data/dota"

pd_match = pd.read_csv(f"{DATA_DIR}/match.csv")
pd_cluster_regions = pd.read_csv(f"{DATA_DIR}/cluster_regions.csv")
pd_purchase_log: pd.DataFrame = pd.read_csv(f"{DATA_DIR}/purchase_log.csv").sample(frac=0.1)
pd_item_id_names = pd.read_csv(f"{DATA_DIR}/item_ids.csv")
pd_players = pd.read_csv(f"{DATA_DIR}/players.csv")

In [119]:
# add region names to match data

match_with_region = pd.merge(pd_match, pd_cluster_regions, how="left", on="cluster").drop(columns="cluster")

In [120]:
# add item names to item purchases

purchases_with_item_names = pd.merge(pd_purchase_log, pd_item_id_names, how="left", on="item_id").drop(columns="item_id")

# get list of timestamps when an item was bought by a player in a match
purchases_with_item_names = purchases_with_item_names.groupby(["match_id", "player_slot", "item_name"])["time"].apply(list).reset_index()

# build a dict with key => item name and value => timestamp list
purchases_with_item_names = purchases_with_item_names.groupby(["match_id", "player_slot"]).apply(lambda x: dict(zip(x["item_name"], x["time"]))).reset_index(name="purchases")

purchases_with_item_names.head(15)

,match_id,player_slot,purchases
0,0,0,"{'arcane_boots': [566], 'blade_of_alacrity': [..."
1,0,1,"{'belt_of_strength': [495], 'blade_of_alacrity..."
2,0,2,"{'arcane_boots': [471], 'blade_of_alacrity': [..."
3,0,3,"{'belt_of_strength': [822, 1339], 'blade_of_al..."
4,0,4,"{'belt_of_strength': [511], 'blade_of_alacrity..."
5,0,128,"{'bfury': [1455], 'blades_of_attack': [1653], ..."
6,0,129,"{'ancient_janggo': [1536], 'blades_of_attack':..."
7,0,130,"{'belt_of_strength': [420], 'blade_of_alacrity..."
8,0,131,"{'arcane_boots': [1736], 'boots': [543], 'bran..."
9,0,132,"{'belt_of_strength': [706], 'black_king_bar': ..."


In [33]:
# add match and item information to player data

player_data = pd_players.query("account_id != 0")
player_data_with_items = pd.merge(player_data, purchases_with_item_names, on=["match_id", "player_slot"])

pd_final = pd.merge(player_data_with_items, match_with_region, how="left", on="match_id")
pd_final = pd_final.dropna(axis=1, thresh=int(0.8 * len(pd_final)))  # drop cols with more than 20% NaN

pd_final.head(15)

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,region
0,0,1,51,1,2954,17760,494,659,13,3,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
1,0,2,11,3,1179,22505,599,605,8,4,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
2,0,3,67,4,3307,23825,613,762,20,3,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
3,0,4,106,128,476,12285,397,524,5,6,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
4,0,5,46,130,2390,13395,452,517,4,8,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
5,0,6,73,132,60,17550,496,456,1,11,...,1982,4,3,63,1,22,True,0,1,SINGAPORE
6,1,7,82,1,9,19625,581,756,9,10,...,0,1846,63,0,221,22,False,0,2,SINGAPORE
7,1,8,39,3,2400,14395,460,544,12,15,...,0,1846,63,0,221,22,False,0,2,SINGAPORE
8,1,4,21,4,1051,12910,365,436,6,11,...,0,1846,63,0,221,22,False,0,2,SINGAPORE
9,1,5,67,131,4055,24165,631,755,16,5,...,0,1846,63,0,221,22,False,0,2,SINGAPORE


### Analyse the data

#### Win rates

In [59]:
pd_final["duration"].describe()


count    307148.000000
mean       2474.469666
std         638.723323
min          59.000000
25%        2025.000000
50%        2409.000000
75%        2870.000000
max       16037.000000
Name: duration, dtype: float64

In [73]:
pd_final["long_game"] = pd_final["duration"] > 4000  # mean + 2.5std
pd_final["short_game"] = pd_final["duration"] < 800  # mean - 2.5std

pd_final.groupby(["long_game", "short_game"])["radiant_win"].mean()
# radiant has a higher win rate in short games

long_game  short_game
False      False         0.516128
           True          0.675522
True       False         0.502946
Name: radiant_win, dtype: float64